In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash import dcc, html
from pandas_datareader import wb
from datetime import datetime, timedelta
from dash_bootstrap_templates import load_figure_template
from dash.dependencies import Input, Output

### Data

Index Data

In [2]:
# Get index tickers and names
tickers =  {
    '^N225'  : 'Nikkei 225',
    '^HSI' : 'Hang Seng',
    '^OMX'  : 'OMX Stockholm 30',
    '^DJI'  : 'Dow Jones',
    '^N100'  : 'Euronext 100',
    '^GSPC'  : 'S&P 500',
    '^NDX' : 'NASDAQ 100',
    'OSEBX.OL' : 'OSEBX'    
}

In [3]:
end = int(datetime.today().timestamp())
start = int((datetime.today() - timedelta(30)).timestamp())

In [4]:
# List for storing temporary dfs
temp = []

# Loop over the tickers
for ticker in tickers.keys():

    # Define url
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' + ticker + '?period1=' + str(start) + '&period2=' + str(end) + '&interval=1d&events=history&includeAdjustedClose=true'

    # Import data from url
    dfTemp = pd.read_csv(url)
    
    # Add ticker as column
    dfTemp['ticker'] = ticker
    
    # Append to list
    temp.append(dfTemp)

In [5]:
# Concat all dfs into single df
df = pd.concat(temp)

# Convert Date to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

print(df['ticker'].nunique())
df.head()

8


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2023-04-19,28619.839844,28677.220703,28531.539063,28606.759766,28606.759766,58000000,^N225
2023-04-20,28472.000000,28694.250000,28442.449219,28657.570313,28657.570313,56700000,^N225
2023-04-21,28590.550781,28778.369141,28527.800781,28564.369141,28564.369141,61400000,^N225
2023-04-24,28631.779297,28680.650391,28567.240234,28593.519531,28593.519531,50000000,^N225
2023-04-25,28697.730469,28806.689453,28609.769531,28620.070313,28620.070313,59300000,^N225


In [6]:
# turn open, high, low, close and adj close to cumulative returns
df['Open'] = df['Open'].pct_change() * 100
df['High'] = df['High'].pct_change() * 100
df['Low'] = df['Low'].pct_change() * 100
df['Close'] = df['Close'].pct_change() * 100

# remove the first row of df
df = df.iloc[1:]

In [7]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2023-05-11,1.032122,0.310309,-0.088357,-0.210132,1201.489990,73814700,OSEBX.OL
2023-05-12,-1.068496,-0.153851,-0.135972,0.967132,1213.109985,69484200,OSEBX.OL
2023-05-15,0.954391,0.448262,0.709237,0.291815,1216.650024,73908800,OSEBX.OL
2023-05-16,0.289299,0.073831,0.204053,-0.467683,1210.959961,67043400,OSEBX.OL
2023-05-18,-0.479136,-0.735296,0.244199,0.000000,1210.959961,4209969419,OSEBX.OL


### Initialising Callbacks

In [8]:
options = [
    {'label' : 'Closing', 'value' : 'Close'},
    {'label' : 'High', 'value' : 'High'},
    {'label' : 'Low', 'value' : 'Low'},
    {'label' : 'Open', 'value' : 'Open'},
]

In [9]:
# Assume you have a list of colors as long as your tickers list
colors = ['red', 'blue', 'green', 'orange', 'purple', 'yellow', 'black', 'pink']

# Create a color map dictionary
color_map = {ticker: color for ticker, color in zip(tickers.keys(), colors)}

### Application

In [10]:
dbcCSS = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css'

In [11]:
app = JupyterDash(external_stylesheets = [dbc.themes.BOOTSTRAP, dbcCSS])

app.layout = dbc.Container(
    children = [
        
        # Header
        html.H1('BAN438 Index Viewer'),
        html.P('A financial data viewer developed in Python and Dash'),
        
        dbc.Row(
            children = [
                ## Radio Buttons
                dbc.Col(
                    dcc.RadioItems(
                        options = options, 
                        value = 'Close',     # component_property for input
                        id = 'my_input',     # component_id for input
                        labelStyle = {'margin-right': '20px'}, # adjust spacing between label and input
                        style = {'fontSize': '15px',
                               'textAlign':'left'} # adjust display                                             
                    ),
                    width = 6,   # half the width of the container
                ),

                ## Currently Viewing: X
                dbc.Col(
                    html.P(
                        children = '',    # component_property for output (the empty string is just a placeholder)          
                        id = 'my_output', # component_id for output, 
                        style = {'fontSize': '15px',
                                 'textAlign': 'centre'} # adjust font size
                    ),
                    width = 6,   # half the width of the container
                ),
            ]
        ),

        dbc.Row(
            children = [
                dbc.Col(
                    # Dropdown menu to select index
                    dcc.Dropdown(
                        options = [{'label' : tickers[ticker], 'value' : ticker} for ticker in tickers.keys()],
                        value = '^N225', # default value
                        id = 'index_dropdown',
                        style = {'fontSize': '15px',
                                'textAlign': 'left',
                                'width': '50%'}
                    ),
                    width = 12,   # half the width of the container
                ),
            ]
        ),

        html.Br(), # line break

        ### Line graph of chosen option of chosen index 3/4th of the container
        dbc.Row(
            children = [
                dbc.Col(
                    dcc.Graph(
                        id = 'main_graph',
                        figure = px.line(df, x = df.index, y = 'Close', color = 'ticker')
                    ),
                    width = 8,
                ),
            ]
        ),

                        
    ],
    
    className = 'dbc',

    style = { # all style elements are written in camelCase, while the properties are written in kebab-case
        'textAlign' : 'left',
        'backgroundColor' : '#BDB7AB',       # change background color using a color string
        'color' : '#363636',               # change text color using a Hex color code
    },
)

@app.callback(
    [Output(component_id='my_output', component_property='children'),  # link the output from the function with html.P
     Output(component_id='main_graph', component_property='figure')],  # link the output from the function with dcc.Graph
    [Input(component_id='my_input', component_property='value'),  # link the input to the function with dcc.RadioItems
     Input(component_id='index_dropdown', component_property='value')]  # link the input to the function with dcc.Dropdown
)
def metricUpdate(choice, selected_ticker):
    # Update the text
    output_text = 'Currently viewing: ' + str(choice) + '.'

    # Update the dataframe based on the selected ticker
    df_filtered = df[df['ticker'] == selected_ticker]

    # Update the graph
    fig = px.line(df_filtered, x=df_filtered.index, y=choice, color='ticker', title = tickers[selected_ticker])

    # Define color map
    color_map = {'^N225': 'red', '^HSI': 'blue', '^OMX': 'green', '^DJI': 'purple', 
                 '^N100': 'red', '^GSPC': 'blue', '^NDX': 'green', 'OSEBX.OL': 'purple'}

    fig.for_each_trace(lambda trace: trace.update(line=dict(color=color_map[trace.name])))

    # Change the background color
    fig.update_layout(
        paper_bgcolor='#BDB7AB',  # this changes the color outside the plot
        plot_bgcolor='#D6CFC7',  # this changes the color of the plot background
        yaxis=dict(range=[-2.5, 6])  # set the Y-axis range here
    )

    return output_text, fig

In [12]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
